In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score 

In [2]:
digits=load_digits()
X,y=digits.data, digits.target 

In [3]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2,random_state=11)

In [4]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.data=torch.tensor(data, dtype=torch.float32)
        self.target=torch.tensor(target, dtype=torch.long)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        sample={'data':self.data[index], 'target':self.target[index]}
        return sample 

In [6]:
train_dataset=CustomDataset(X_train, y_train)
test_dataset=CustomDataset(X_test, y_test)

In [7]:
train_dataset[1]

{'data': tensor([ 0.0000, -0.3266, -0.2477,  0.9870, -1.3667, -1.0131, -0.4107, -0.1187,
         -0.0558, -0.6240,  0.2959,  0.7518, -1.9585, -1.3309, -0.5054, -0.1243,
         -0.0354, -0.4485,  1.0672,  0.6793, -1.1479, -1.2392, -0.5341, -0.1049,
         -0.0373,  1.7479,  1.1191, -1.0058, -1.6104, -0.0810,  0.4738, -0.0264,
          0.0000,  2.7501,  1.3089, -0.5060,  0.1192,  1.2237,  1.1576,  0.0000,
         -0.0577,  1.8022,  1.3734,  1.3643,  1.1887, -0.9234, -0.8084, -0.0940,
         -0.0334, -0.4056, -0.6385,  1.2375,  0.1230, -1.4744, -0.7676, -0.2048,
         -0.0264, -0.2901, -0.2996,  0.8951, -1.1972, -1.1653, -0.4980, -0.1942]),
 'target': tensor(4)}

In [8]:
train_dataset

In [9]:
train_data_loader=DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_data_loader=DataLoader(dataset=test_dataset, batch_size=32, shuffle=True, num_workers=4)

In [10]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1=nn.Linear(input_size, hidden_size)
        self.relu=nn.ReLU()
        self.layer2=nn.Linear(hidden_size,output_size)
    
    def forward(self, x):
        x=self.layer1(x)
        x=self.relu(x)
        x=self.layer2(x)
        return x 

In [11]:
input_size=X_train.shape[1]
hidden_size=64
output_size=len(set(y_train))

In [12]:
model=SimpleNN(input_size,hidden_size, output_size)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [13]:
num_epochs=100
for epoch in range(num_epochs):
    model.train()
    running_loss=0.0

    for batch in train_data_loader:
        inputs=batch['data']
        targets=batch['target']

        optimizer.zero_grad()
        outputs=model(inputs)

        loss=criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_data_loader)}")

Epoch 1/100, Loss: 1.8890548096762763
Epoch 2/100, Loss: 1.078640541765425
Epoch 3/100, Loss: 0.5805997755792406
Epoch 4/100, Loss: 0.3595220741298464
Epoch 5/100, Loss: 0.2540568480889002
Epoch 6/100, Loss: 0.1923285656505161
Epoch 7/100, Loss: 0.1550344631075859
Epoch 8/100, Loss: 0.1270064910252889
Epoch 9/100, Loss: 0.10683407870431741
Epoch 10/100, Loss: 0.09092842845453156
Epoch 11/100, Loss: 0.078968598610825
Epoch 12/100, Loss: 0.06893792202075323
Epoch 13/100, Loss: 0.06061387418044938
Epoch 14/100, Loss: 0.05384456130365531
Epoch 15/100, Loss: 0.04796577468514442
Epoch 16/100, Loss: 0.043197278802593546
Epoch 17/100, Loss: 0.03903482494254907
Epoch 18/100, Loss: 0.03497828340364827
Epoch 19/100, Loss: 0.03167054255803426
Epoch 20/100, Loss: 0.029110124727918043
Epoch 21/100, Loss: 0.02633553436025977
Epoch 22/100, Loss: 0.02403936701723271
Epoch 23/100, Loss: 0.022143109432525104
Epoch 24/100, Loss: 0.02021323822231756
Epoch 25/100, Loss: 0.018780576644672288
Epoch 26/100, Lo

In [16]:
model.eval()
all_predictions=[]
all_targets=[]

with torch.no_grad():
    for batch in test_data_loader:
        inputs=batch['data']
        targets=batch['target']

        outputs=model(inputs)
        predictions=torch.argmax(outputs, dim=1)

        all_predictions.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
    
accuracy=accuracy_score(all_targets,all_predictions)
print(f"Accuracy Test: {accuracy * 100: .3f}%")



Accuracy Test:  98.333%
